In [2]:
from dotenv import load_dotenv , find_dotenv
import os
dotenv_path = find_dotenv()

load_dotenv(find_dotenv())
api_key = os.getenv("OPENAI_API_KEY")

if api_key is None:
    raise ValueError("OPENAI_API_KEY not found in environment variables")






In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
#Wiki tool 
wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)
wiki_tool.name

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://python.langchain.com/v0.1/docs/modules/tools/")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
# Embedding 
#Using FAISS vector DB
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(chunks, embedding=embeddings)  # Pass embeddings as parameter
retriever = db.as_retriever()
retriever
# print(f"Total chunks indexed: {db.index.ntotal}")
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langchain_tool",
                                       "Search for information about langchain tools. For any questions about Langchain tools , you must use this tool.")

retriever_tool.name

'langchain_tool'

In [5]:
#Arxiv tool 
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=100)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name
tools = [wiki_tool, arxiv_tool, retriever_tool]
tools


[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\python\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=100)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=100, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langchain_tool', description='Search for information about langchain tools. For any questions about Langchain tools , you must use this tool.', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000226D9224180>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langc

In [6]:
from langchain import OpenAI
llm = ChatOpenAI(model = "gpt-3.5-turbo-0125", temperature = 0.3)

#langsmith hub - for prompts
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\python\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [8]:

#Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)


In [9]:
#Executing agents 
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor
agent_executor.invoke({"input": "Tell me about Langchain Tools"})



> Entering new AgentExecutor chain...

Invoking: `langchain_tool` with `{'query': 'Langchain Tools'}`


Tools | 🦜️🔗 LangChain

Skip to main contentLangChain v0.2 is out! You are currently viewing the old v0.1 docs. View the latest docs here.ComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTubev0.1v0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchModel I/OPromptsChat modelsLLMsOutput parsersRetrievalDocument loadersText splittersEmbedding modelsVector storesRetrieversIndexingCompositionToolsToolkitsToolsDefining Custom ToolsTools as OpenAI FunctionsAgentsChainsMoreComponentsCompositionToolsOn this pageToolsTools are interfaces that an agent, chain, or LLM can use to interact with the world.

up things in wikipedia",    args_schema=WikiInputs,    api_wrapper=api_wrapper,    return_direct=True,)tool.name'wiki-tool'tool.description'look up things in wikipedia'tool.args{'query'

{'input': 'Tell me about Langchain Tools',
 'output': 'Langchain Tools are interfaces that an agent, chain, or LLM can use to interact with the world. Langchain is a framework designed to simplify the creation of applications. It offers various components, integrations, guides, API references, and more to assist in the development process. Langchain Tools can be used to define custom tools, work with toolkits, and interact with the world effectively. For more detailed information, you can refer to the LangChain documentation.'}